In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import FodorsZagatsBenchmark

benchmark = FodorsZagatsBenchmark(data_dir_path="../data/")
benchmark

14:52:38 INFO:Extracting Fodors-Zagats...
14:52:38 INFO:Reading Fodors-Zagats row_dict...
14:52:38 INFO:Reading Fodors-Zagats train.csv...
14:52:38 INFO:Reading Fodors-Zagats valid.csv...
14:52:38 INFO:Reading Fodors-Zagats test.csv...


<FodorsZagatsBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Fodors-Zagats/fodors_zagat_exp_data.zip

## Preprocess

In [5]:
attr_list = ['name', 'addr', 'city', 'phone', 'type']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

  0%|          | 0/864 [00:00<?, ?it/s]

### Rename attr `type` to `type_` (avoids clash on pytorch)

In [7]:
for row in tqdm(benchmark.row_dict.values()):
    row['type_'] = row['type']
    del row['type']

  0%|          | 0/864 [00:00<?, ?it/s]

## Init Data Module

In [8]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [9]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [10]:
attr_config_dict = {
    'name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'addr': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'city': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'phone': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'type_': {
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
}

In [11]:
from entity_embed import AttrConfigDictParser

row_numericalizer = AttrConfigDictParser.from_dict(attr_config_dict, row_list=benchmark.row_dict.values())
row_numericalizer.attr_config_dict

14:52:38 INFO:For attr=name, computing actual max_str_len
14:52:38 INFO:For attr=name, using actual_max_str_len=12
14:52:38 INFO:For attr=addr, computing actual max_str_len
14:52:38 INFO:actual_max_str_len=13 must be even to enable NN pooling. Updating to 14
14:52:38 INFO:For attr=addr, using actual_max_str_len=14
14:52:38 INFO:For attr=city, computing actual max_str_len
14:52:38 INFO:For attr=city, using actual_max_str_len=10
14:52:38 INFO:For attr=phone, computing actual max_str_len
14:52:38 INFO:For attr=phone, using actual_max_str_len=18
14:52:38 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt


{'name': AttrConfig(source_attr='name', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=12, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True),
 'addr': AttrConfig(source_attr='addr', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+'

In [12]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=16,
    eval_batch_size=128,
    random_seed=random_seed
)

## Training

In [13]:
from entity_embed import Matcher

model = Matcher(
    row_numericalizer=row_numericalizer,
)

In [14]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStoppingMinEpochs(
   monitor='valid_f1_at_0.5',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_save_dir = '../tb_logs'
tb_name = f'matcher-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_save_dir, name=tb_name),
    reload_dataloaders_every_epoch=True
)

GPU available: True, used: True
14:52:41 INFO:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
14:52:41 INFO:TPU available: None, using: 0 TPU cores


In [15]:
trainer.fit(model, datamodule)

14:52:41 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type              | Params
--------------------------------------------------
0 | matcher_net | MatcherNet        | 6.1 M 
1 | loss_fn     | BCEWithLogitsLoss | 0     
--------------------------------------------------
6.1 M     Trainable params
32.7 K    Non-trainable params
6.1 M     Total params
24.540    Total estimated model params size (MB)
14:52:44 INFO:
  | Name        | Type              | Params
--------------------------------------------------
0 | matcher_net | MatcherNet        | 6.1 M 
1 | loss_fn     | BCEWithLogitsLoss | 0     
--------------------------------------------------
6.1 M     Trainable params
32.7 K    Non-trainable params
6.1 M     Total params
24.540    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

14:55:04 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'test_f1_at_0.3': 0.8936169743537903,
  'test_f1_at_0.4': 0.9130434393882751,
  'test_f1_at_0.5': 0.9130434393882751,
  'test_precision_at_0.3': 0.8399999737739563,
  'test_precision_at_0.4': 0.875,
  'test_precision_at_0.5': 0.875,
  'test_recall_at_0.3': 0.9545454382896423,
  'test_recall_at_0.4': 0.9545454382896423,
  'test_recall_at_0.5': 0.9545454382896423}]